In [ ]:
import os
import requests
import time
from dotenv import load_dotenv
load_dotenv()


: 

In [7]:
# Read credentials from environment variables
# Note: Device Code flow doesn't require username/password - authentication happens in browser
DATAVERSE_TENANT_ID = os.environ.get('DATAVERSE_TENANT_ID')
DATAVERSE_CLIENT_ID = os.environ.get('DATAVERSE_CLIENT_ID')
DATAVERSE_HOST = os.environ.get('DATAVERSE_HOST')

# Verify required credentials are set
if not DATAVERSE_TENANT_ID:
    print("Error: DATAVERSE_TENANT_ID environment variable not set")
    exit(1)
if not DATAVERSE_CLIENT_ID:
    print("Error: DATAVERSE_CLIENT_ID environment variable not set")
    exit(1)
if not DATAVERSE_HOST:
    print("Error: DATAVERSE_HOST environment variable not set")
    exit(1)

In [ ]:
# Get OAuth token using Device Code flow (supports MFA)


device_code_url = f"https://login.microsoftonline.com/{DATAVERSE_TENANT_ID}/oauth2/v2.0/devicecode"
device_code_response = requests.post(device_code_url, data={
    'client_id': DATAVERSE_CLIENT_ID,
    'scope': f'{DATAVERSE_HOST}/.default'
})
device_code_data = device_code_response.json()

print(device_code_data['message'])

device_code = device_code_data['device_code']
token_url = f"https://login.microsoftonline.com/{DATAVERSE_TENANT_ID}/oauth2/v2.0/token"

while True:
    token_response = requests.post(token_url, data={
        'grant_type': 'urn:ietf:params:oauth:grant-type:device_code',
        'client_id': DATAVERSE_CLIENT_ID,
        'device_code': device_code
    })
    token_data = token_response.json()
    
    if 'access_token' in token_data:
        token = token_data['access_token']
        print("\nAuthentication successful!")
        break
    
    time.sleep(device_code_data.get('interval', 5))

In [4]:
# Token is already extracted in the previous cell
# Verify we have a token
if 'token' not in locals() or not token:
    print("Error: Failed to obtain access token")
    exit(1)

print(f"Token obtained successfully (length: {len(token)} characters)")

Token obtained successfully (length: 2388 characters)


In [ ]:
# Query DataVerse metadata API to list all tables
headers = {'Authorization': f'Bearer {token}'}

metadata_url = f"{DATAVERSE_HOST}/api/data/v9.2/EntityDefinitions?$select=LogicalName,SchemaName,DisplayName"
response = requests.get(metadata_url, headers=headers)

if response.status_code == 200:
    tables = response.json()['value']
    print(f"\nFound {len(tables)} tables:\n")
    for table in sorted(tables, key=lambda x: x['LogicalName']):
        logical_name = table['LogicalName']
        
        # Safely get display name
        display_name = 'N/A'
        if 'DisplayName' in table and table['DisplayName']:
            if 'UserLocalizedLabel' in table['DisplayName'] and table['DisplayName']['UserLocalizedLabel']:
                display_name = table['DisplayName']['UserLocalizedLabel'].get('Label', 'N/A')
        
        print(f"{logical_name:<40}\t\t{display_name}")
else:
    print(f"Error {response.status_code}: {response.text}")

In [ ]:
# Fetch sample data from aibs_fact_mouse_weights_water_log table for exploration
data_url = f"{DATAVERSE_HOST}/api/data/v9.2/aibs_fact_mouse_weights_water_logs?$top=2"

data_response = requests.get(data_url, headers=headers)

if data_response.status_code == 200:
    data = data_response.json()
    rows = data.get('value', [])
    print(f"Found {len(rows)} records. Showing full entries:\n")
    
    for i, row in enumerate(rows, 1):
        print(f"Record {i}:")
        for key, value in row.items():
            print(f"  {key}: {value}")
        print()
else:
    print(f"Error {data_response.status_code}: {data_response.text}")

In [13]:
# Fetch data with expanded mouse relationship - just 2 records
data_url = f"{DATAVERSE_HOST}/api/data/v9.2/aibs_fact_mouse_weights_water_logs?$top=2&$expand=aibs_mouse_id($select=aibs_mouse_id,aibs_name)"

data_response = requests.get(data_url, headers=headers)

if data_response.status_code == 200:
    data = data_response.json()
    rows = data.get('value', [])
    print(f"\nFound {len(rows)} rows:\n")
    
    for i, row in enumerate(rows, 1):
        print(f"Row {i}:")
        # Show the expanded mouse data
        if 'aibs_mouse_id' in row:
            mouse = row['aibs_mouse_id']
            print(f"  Mouse ID: {mouse.get('aibs_mouse_id', 'N/A')}")
            print(f"  Mouse Name: {mouse.get('aibs_name', 'N/A')}")
        print(f"  Weight: {row.get('aibs_weight')}")
        print(f"  Date/Time: {row.get('aibs_date_time')}")
        print(f"  Water Total: {row.get('aibs_water_total')}")
        print()
else:
    print(f"Error {data_response.status_code}: {data_response.text}")

Error 400: {"error":{"code":"0x80060888","message":"Could not find a property named 'aibs_name' on type 'Microsoft.Dynamics.CRM.aibs_dim_mice'."}}


In [ ]:
# Fetch the mouse record by its GUID
mouse_guid = "62f62c7f-dd89-f011-b4cb-6045bd03524b"
mouse_url = f"{DATAVERSE_HOST}/api/data/v9.2/aibs_dim_mices({mouse_guid})"

mouse_response = requests.get(mouse_url, headers=headers)

if mouse_response.status_code == 200:
    mouse = mouse_response.json()
    print("Mouse record:")
    for key, value in mouse.items():
        print(f"  {key}: {value}")
else:
    print(f"Error {mouse_response.status_code}: {mouse_response.text}")

In [ ]:
# Fetch sample data from aibs_fact_team_behavior_training
data_url = f"{DATAVERSE_HOST}/api/data/v9.2/aibs_fact_team_behavior_trainings?$top=3"

data_response = requests.get(data_url, headers=headers)

if data_response.status_code == 200:
    data = data_response.json()
    rows = data.get('value', [])
    print(f"\nFound {len(rows)} rows from aibs_fact_team_behavior_training:\n")
    
    if rows:
        # Print first row's keys to see what columns exist
        print("Columns:", list(rows[0].keys()))
        print("\n")
        
        # Print each row
        for i, row in enumerate(rows, 1):
            print(f"Row {i}:")
            for key, value in row.items():
                if not key.startswith('@') and not key.startswith('_owning'):
                    print(f"  {key}: {value}")
            print()
    else:
        print("No rows found")
else:
    print(f"Error {data_response.status_code}: {data_response.text}")

In [ ]:
# Fetch sample data from aibs_dim_mouse_workflows
data_url = f"{DATAVERSE_HOST}/api/data/v9.2/aibs_dim_mouse_workflowses?$top=3"

data_response = requests.get(data_url, headers=headers)

if data_response.status_code == 200:
    data = data_response.json()
    rows = data.get('value', [])
    print(f"\nFound {len(rows)} rows from aibs_dim_mouse_workflows:\n")
    
    if rows:
        print("Columns:", list(rows[0].keys()))
        print("\n")
        
        for i, row in enumerate(rows, 1):
            print(f"Row {i}:")
            for key, value in row.items():
                if not key.startswith('@') and not key.startswith('_owning'):
                    print(f"  {key}: {value}")
            print()
    else:
        print("No rows found")
else:
    print(f"Error {data_response.status_code}: {data_response.text}")

In [ ]:
# Fetch sample data from aibs_dim_mouse_workflow_steps
data_url = f"{DATAVERSE_HOST}/api/data/v9.2/aibs_dim_mouse_workflow_stepses?$top=5"

data_response = requests.get(data_url, headers=headers)

if data_response.status_code == 200:
    data = data_response.json()
    rows = data.get('value', [])
    print(f"\nFound {len(rows)} rows from aibs_dim_mouse_workflow_steps:\n")
    
    if rows:
        print("Columns:", list(rows[0].keys()))
        print("\n")
        
        for i, row in enumerate(rows, 1):
            print(f"Row {i}:")
            for key, value in row.items():
                if not key.startswith('@') and not key.startswith('_owning'):
                    print(f"  {key}: {value}")
            print()
    else:
        print("No rows found")
else:
    print(f"Error {data_response.status_code}: {data_response.text}")

In [ ]:
# Query with filters and field selection, including mouse ID
table_name = "aibs_fact_mouse_weights_water_logs"
data_url = f"{DATAVERSE_HOST}/api/data/v9.2/{table_name}?$filter=aibs_weight gt 20&$select=aibs_weight,aibs_date_time&$expand=aibs_mouse_id($select=aibs_mouse_id)&$top=5"
response = requests.get(data_url, headers=headers)

if response.status_code == 200:
    data = response.json()
    rows = data.get('value', [])
    print(f"Found {len(rows)} records with weight > 20g:\n")
    
    for i, row in enumerate(rows, 1):
        date_time = row.get('aibs_date_time', 'N/A')
        if date_time and date_time != 'N/A':
            date_time = date_time.replace('T', ' ').replace('Z', ' UTC')
        
        # Get mouse ID from expanded relationship
        mouse_id = 'N/A'
        if 'aibs_mouse_id' in row and row['aibs_mouse_id']:
            mouse_id = row['aibs_mouse_id'].get('aibs_mouse_id', 'N/A')
        
        print(f"Record {i}:")
        print(f"  Mouse ID: {mouse_id}")
        print(f"  Weight: {row.get('aibs_weight', 'N/A')} g")
        print(f"  Date/Time: {date_time}")
        print()
else:
    print(f"Error {response.status_code}: {response.text}")